In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
import os
os.chdir('/home/habte/contract_QA_Rag_project')  
from src.pdf_loader import PDFLoader
from src.vector_embedding import VectorEmbedding
from src.chat_model import ChatModel

In [3]:
if __name__ == "__main__":
    # Define paths to PDF documents
    pdf_paths = [
        "/home/habte/Downloads/Raptor Contract.docx.pdf",
        "/home/habte/Downloads/Raptor Q&A2.docx.pdf",
        "/home/habte/Downloads/Robinson Advisory.docx.pdf",
        "/home/habte/Downloads/Robinson Q&A.docx.pdf"
    ]

    openai_api_key = os.environ.get('OPENAI_API_KEY')

    # Initialize PDF loader
    pdf_loader = PDFLoader(pdf_paths)
    split_data = pdf_loader.load_and_split_documents()

    # Initialize vector embedding
    vector_embedding = VectorEmbedding(openai_api_key)
    vect_db = vector_embedding.create_vector_store(split_data)

    # Initialize chat model
    chat_model = ChatModel(openai_api_key)
    chat_qa = chat_model.create_chat_qa(vect_db)


/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
chat_history = []

while True:
    query = input('You: ')  # Prompt the user to input a question

    if query.lower() == 'done':  
        break

    response = chat_qa({"question": query, "chat_history": chat_history})  # Retrieve response

    chat_history.append({"role": "user", "content": query})  # Update chat history
    chat_history.append({"role": "assistant", "content": response["answer"]})

    print('Assistant:', response["answer"])  # Print the assistant's response

In [42]:
# manual evaluation and debugging
user_questions = [
    "Who are the parties to the Agreement and what are their defined names?",
    "What are the payments to the Advisor under the Agreement?",
    "Is there a non-compete obligation to the Advisor?"
]

In [43]:
for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")
    

User: Who are the parties to the Agreement and what are their defined names?
RAG System Response: The parties to the Agreement are not explicitly mentioned in the provided context, so it is unclear who they are and what their defined names are.
Debugging Information:
User: What are the payments to the Advisor under the Agreement?
RAG System Response: According to section 6 of the Agreement, the payments to the Advisor are as follows:
1. Fees of $9 per hour up to a monthly limit of $1,500.
2. Workspace expense of $100 per month.
3. Other reasonable and actual expenses if approved by the company in writing and in advance.
Debugging Information:
User: Is there a non-compete obligation to the Advisor?
RAG System Response: Yes, there is a non-compete obligation for the Advisor mentioned in the agreement. The Advisor is required to execute a Confidentiality, Non-Competition, and IP Ownership Undertaking as part of their obligations under the Agreement.
Debugging Information:


In [41]:
user_questions = [
    "How much is the escrow amount?",
    "What is the purpose of the escrow?",
    "Whose consent is required for the assignment of the Agreement by the Buyer?"
]

for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")

User: How much is the escrow amount?
RAG System Response: The escrow amount is equal to $1,000,000.
Debugging Information:
User: What is the purpose of the escrow?
RAG System Response: The purpose of an escrow account is to hold funds, assets, or documents on behalf of two parties involved in a transaction until specified conditions are met. In this context, the escrow is used for holding funds related to the purchase price adjustment provisions and terms of the agreement until disbursements are made based on the agreement's requirements.
Debugging Information:
User: Whose consent is required for the assignment of the Agreement by the Buyer?
RAG System Response: The consent required for the assignment of the Agreement by the Buyer is not explicitly mentioned in the provided context.
Debugging Information:


In [44]:
from langchain_community.chat_models import ChatOpenAI


openai_key = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Initialize the separate language model
separate_llm = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")

# Now you can proceed with the rest of your code
for question in user_questions:
    # Generate response using the RAG system
    rag_response = chat_qa({"question": question, "chat_history": []})

    # Generate response using the separate language model
    separate_llm_response = separate_llm.invoke(question)

    # Print the user's question and both responses for comparison
    print("User:", question)
    print("RAG System Response:", rag_response["answer"])
    print("Separate LLM Response:", separate_llm_response)

    # Compare the responses and identify discrepancies
    if rag_response["answer"] != separate_llm_response:
        print("Discrepancy found!")


User: Who are the parties to the Agreement and what are their defined names?
RAG System Response: The parties to the Agreement are not explicitly stated in the provided context, therefore, I do not know their defined names.
Separate LLM Response: content='The parties to the Agreement are typically referred to as Party A and Party B. Their defined names can vary depending on the specific agreement and the entities involved. Party A is usually the party that initiates the agreement or makes an offer, while Party B is the party who accepts the offer or enters into the agreement with Party A. The defined names of the parties could be the names of individuals, companies, organizations, or other entities involved in the agreement.'
Discrepancy found!
User: What are the payments to the Advisor under the Agreement?
RAG System Response: According to section 6 of the Agreement, the payments to the Advisor include:
1. Fees of $9 per hour up to a monthly limit of $1,500,
2. Workspace expense of $1

In [38]:
from langchain_community.chat_models import ChatOpenAI
from nltk.translate.bleu_score import sentence_bleu

openai_key = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Initialize the separate language model
separate_llm = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")

# Lists to store responses and BLEU scores
rag_responses = []
separate_llm_responses = []
bleu_scores = []


for question in user_questions:
    # Generate response using the RAG system
    rag_response = chat_qa({"question": question, "chat_history": []})["answer"]
    rag_responses.append(rag_response)

    # Generate response using the separate language model
    separate_llm_response = separate_llm.invoke(question)
    separate_llm_responses.append(separate_llm_response.content)  # Extracting text from the AIMessage object

    # Compute BLEU score for the response pair
    bleu_score = sentence_bleu([rag_response], separate_llm_response.content)
    bleu_scores.append(bleu_score)

    # Print the user's question and both responses for comparison
    print("User:", question)
    print("RAG System Response:", rag_response)
    print("Separate LLM Response:", separate_llm_response.content)

    # Compare the responses and identify discrepancies
    if rag_response != separate_llm_response.content:
        print("Discrepancy found!")

# Calculate average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", avg_bleu_score)


User: Who are the parties to the Agreement and what are their defined names?
RAG System Response: The parties to the Agreement are Cloud Investments Ltd. ("Company") and Jack Robinson ("Advisor").
Separate LLM Response: The parties to the Agreement are [Party 1] and [Party 2], with their defined names as follows:

- [Party 1]: [Legal Name of Party 1]
- [Party 2]: [Legal Name of Party 2]
Discrepancy found!
User: What are the payments to the Advisor under the Agreement?
RAG System Response: According to section 6 of the Agreement, the payments to the Advisor are as follows:
1. Fees of $9 per hour up to a monthly limit of $1,500.
2. Workspace expense of $100 per month.
3. Other reasonable and actual expenses if approved by the company in writing and in advance.
Separate LLM Response: The payments to the Advisor under the Agreement will typically include a base fee or retainer, as well as additional fees for specific services or transactions. The exact payment structure will vary depending

In [26]:
from langchain_community.chat_models import ChatOpenAI
from nltk.translate.bleu_score import sentence_bleu

openai_key = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Initialize the separate language model with temperature
temperature = 1  # Set the temperature value
separate_llm = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo", temperature=temperature)

# Lists to store responses and BLEU scores
rag_responses = []
separate_llm_responses = []
bleu_scores = []


for question in user_questions:
    # Generate response using the RAG system
    rag_response = chat_qa({"question": question, "chat_history": []})["answer"]
    rag_responses.append(rag_response)

    # Generate response using the separate language model with temperature
    separate_llm_response = separate_llm.invoke(question)
    separate_llm_responses.append(separate_llm_response.content)  # Extracting text from the AIMessage object

    # Compute BLEU score for the response pair
    bleu_score = sentence_bleu([rag_response], separate_llm_response.content)
    bleu_scores.append(bleu_score)

    # Print the user's question and both responses for comparison
    print("User:", question)
    print("RAG System Response:", rag_response)
    print("Separate LLM Response:", separate_llm_response.content)

    # Compare the responses and identify discrepancies
    if rag_response != separate_llm_response.content:
        print("Discrepancy found!")

# Calculate average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", avg_bleu_score)


User: How much is the escrow amount?
RAG System Response: The purpose of the escrow in this context is to hold funds that may be subject to adjustments in the purchase price as outlined in the agreement. The escrow acts as a neutral party that can make disbursements based on the agreed terms of the agreement, especially in cases where there are disputes between the parties involved. Additionally, it serves as a mechanism for resolving any potential issues related to adjustments in the purchase price.
Separate LLM Response: The escrow amount can vary depending on the transaction or agreement in question. It is typically a percentage of the total sale price or a set amount agreed upon by both parties. It is held in a neutral account by a third party until the conditions of the agreement are met.
Discrepancy found!
User: What is the purpose of the escrow?
RAG System Response: The purpose of the escrow in this context is to hold funds in accordance with the purchase price adjustment provis

In [ ]:
import time
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from src.chat_model import ChatModel
from utils.evaluation_metrics import ChatModelTester

openai_key = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Instantiate ChatModel
chat_model = ChatModel(openai_api_key)

# Create chat Q&A instance
chat_qa = chat_model.create_chat_qa(vect_db)

# Initialize ChatModelTester with the chat Q&A instance
tester = ChatModelTester(chat_qa)

# Define test data
test_data = [
    ("Who are the parties to the Agreement and what are their defined names?", "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"),
    ("Is there a non-compete obligation to the Advisor?", "Yes. During the term of engagement with the Company and for a period of 12 months thereafter."),
    ("Whose consent is required for the assignment of the Agreement by the Buyer?", "The consent of both parties is required.")
]

# Test the ChatModel using the defined test data
tester.test_with_dataset(test_data)


In [20]:
# Import the necessary modules
from src.chat_model import ChatModel
from utils.human_evaluation import ChatModelTester

openai_key = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Instantiate ChatModel
chat_model = ChatModel(openai_api_key)

# Create chat Q&A instance
chat_qa = chat_model.create_chat_qa(vect_db)

# Initialize ChatModelTester with the chat Q&A instance
tester = ChatModelTester(chat_qa)

# Define test data
test_data = [
    ("Who are the parties to the Agreement and what are their defined names?", "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"),
    ("How much is the escrow amount?", "The escrow amount is equal to $1,000,000."),
    ("Is escrow amount grete then the Retention Amount ?", "No.")
]

# Test the ChatModel using the defined test data
tester.test_with_dataset(test_data)


Generated Answer: The parties to the Agreement are Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”).
Expected Answer: Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)
Generated Answer: The escrow amount is $1,000,000.
Expected Answer: The escrow amount is equal to $1,000,000.
Generated Answer: No.
Expected Answer: No.
Total Questions: 3
Correct Answers: 3
Accuracy: 100.00%


In [ ]:
# Import the necessary modules
import tensorflow_hub as hub
import numpy as np
from src.chat_model import ChatModel
from utils.semantic_similarity import ChatModelTester

tester = ChatModelTester(chat_qa)
test_data = [
    ("Who are the parties to the Agreement and what are their defined names?", "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"),
    ("Is there a non-compete obligation to the Advisor?", "Yes. During the term of engagement with the Company and for a period of 12 months thereafter."),
    ("Whose consent is required for the assignment of the Agreement by the Buyer?", "The consent of both parties is required.")
]
tester.test_with_dataset(test_data)